# 기업의 데이터이기 때문에 대외비(매출, 매출 그래프, 정확한 수치, 그래프의 X축, Y축 등)는 비식별화 or 제거 처리됨

In [ ]:
# 경고메시지 무시
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 주피터노트북 matplotlib 한글 폰트 깨짐 방지
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False
f_path = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager .FontProperties(fname=f_path).get_name()
rc('font', family=font_name)

In [ ]:
import pandas as pd

# * 주문 데이터

## 데이터 불러오기

In [ ]:
order_df = pd.read_excel('주문DATA_20220601_20230430_map.xlsx')
order_df.head()

## 결측치 확인

In [ ]:
order_df.describe()

In [ ]:
# 결측치 확인
order_df.info()
order_df.isnull().sum()

## 데이터 전처리

In [ ]:
# 필요한 컬럼만 추출
df = order_df[['소매명', '사입업체명', '지역명', '대납비', '처리버튼', '사입메모', '사입처리자', '주문일시', '외상여부', 'crd_pay_dt']]
df.head()

In [ ]:
# 처리버튼이 대납, 결, 교미, 올미, 외상인 값만 추출
df = df[df['처리버튼'].str.contains('대납|결|교미|올미|외상')]
df.head()

In [ ]:
# crd_pay_dt의 null값 -> 142,076개
df.isnull().sum()

소매명                0
사입업체명              0
지역명                0
대납비                0
처리버튼               0
사입메모              25
사입처리자              0
주문일시               0
외상여부               0
crd_pay_dt    142076
dtype: int64

In [ ]:
# crd_pay_dt의 null 값 알아보기
# 외상여부 = N인 row -> 142,070개
df[df['외상여부'] == 'N']

In [ ]:
# 외상여부 = Y & crd_pay_dt가 NULL인 row -> 6개 (아직 외상 납입이 안된 걸로 판단됨)
df[(df['외상여부'] == 'Y') & (df['crd_pay_dt'].isnull())]

In [ ]:
# 정산데이터 - 청구금액은 억대, 입금액은 0원인 데이터가 지속 -> 이상치로 판단되어 제거
df = df.drop(df[df['사입업체명'] == 'B9'].index)

In [ ]:
# 지역별로 외상여부가 Y인 값의 대납비 합계 확인
df[df['외상여부'] == 'Y'].groupby('지역명')['대납비'].sum()  # 광주 -> 0원 / 데이터 확인해보니 주문취소건

In [ ]:
# 광주 & 외상여부 = 'Y' 컬럼 삭제
df = df.drop(df[(df['외상여부'] == 'Y') & (df['지역명'] == '광주')].index)

## 데이터 저장

In [ ]:
df.to_csv('주문데이터_final.csv')

## 시각화

In [ ]:
# 사입업체별로 외상여부가 Y인 값의 대납비 합계 확인 -> null값 없음
df_g = df[df['외상여부'] == 'Y'].groupby('사입업체명')['대납비'].sum().sort_values(ascending = False)
df_g

In [ ]:
# 사입업체별 외상대납비 그래프
# B1 업체만 눈에 띄게 값이 큰 것 확인 -> B1은 제주 업체 -> B1이 대표성을 띌 수 있어서 이상치로 간주하고 제외
plt.figure(figsize=(8,8))

df_g.plot.bar()
plt.xlabel('사입업체명')
plt.ylabel('대납비')
plt.title('사입업체별 외상대납비')
plt.show()

In [ ]:
# B1 제외
df = df.drop(df[df['사입업체명'] == 'B1'].index)

In [ ]:
# B1 제외 후 사입업체별로 외상여부가 Y인 값의 대납비 합계 재확인
df_g = df[df['외상여부'] == 'Y'].groupby('사입업체명')['대납비'].sum().sort_values(ascending = False)
df_g

In [ ]:
# 사입업체별 색깔 입력
colors = ['lightcoral', 'mediumseagreen', 'gold', 'lightskyblue', 'mediumpurple']

plt.figure(figsize=(8,8))

df_g.plot.bar(color=colors)
plt.xlabel('사입업체명')
plt.ylabel('대납비')
plt.title('사입업체별 외상대납비')
plt.show()

In [ ]:
df_a = df.groupby('사입업체명')['대납비'].sum().sort_values(ascending = False)
df_a

In [ ]:
# 모든 사입업체의 총 대납비 그래프 확인

# 지정 bar color 지정
colors = []
for x in df_a.index:
    if x == 'B2':
        colors.append('lightcoral')
    elif x == 'B3':
        colors.append('mediumseagreen')
    elif x == 'B4':
        colors.append('gold')
    elif x == 'B5':
        colors.append('lightskyblue')
    elif x == 'B7':
        colors.append('mediumpurple')
    else:
        colors.append('gray')


plt.figure(figsize=(14,8))

df_a.plot.bar(color=colors)
plt.xlabel('사입업체명')
plt.ylabel('대납비')
plt.title('사입업체별 외상대납비')
plt.show()

In [ ]:
삼촌_외상 = df[df['외상여부'] == 'Y'].groupby('사입처리자')['대납비'].sum().sort_values(ascending = False)
삼촌_외상

사입처리자
E117     211455000
E230      83449000
E55       38173000
E106      22272000
E248        651000
E142        628000
E1003       389000
E302        315000
Name: 대납비, dtype: int64

In [ ]:
# 외상 경험이 있는 사입삼촌의 외상대납비 그래프
colors = ['lightcoral', 'mediumseagreen', 'gold', 'lightskyblue', 'mediumpurple', 'red', 'orange', 'lightpink']

plt.figure(figsize=(8,8))

삼촌_외상.plot.bar(color=colors)
plt.xlabel('사입삼촌')
plt.ylabel('대납비')
plt.title('사입삼촌별 외상대납비')
plt.show()

In [ ]:
삼촌_대납 = df.groupby('사입처리자')['대납비'].sum().sort_values(ascending = False)
삼촌_대납

In [ ]:
# 모든 사입삼촌별 총 대납비 그래프

# 지정 bar color 지정
colors = []
for x in 삼촌_대납.index:
    if x == 'E117':
        colors.append('lightcoral')
    elif x == 'E230':
        colors.append('mediumseagreen')
    elif x == 'E55':
        colors.append('gold')
    elif x == 'E106':
        colors.append('lightskyblue')
    elif x == 'E248':
        colors.append('mediumpurple')
    elif x == 'E142':
        colors.append('red')
    elif x == 'E1003':
        colors.append('orange')
    elif x == 'E302':
        colors.append('lightpink')

    else:
        colors.append('gray')


plt.figure(figsize=(14,8))

삼촌_대납.plot.bar(color=colors)
plt.xlabel('사입삼촌')
plt.ylabel('대납비')
plt.title('사입삼촌별 총 대납비')
plt.show()

---

# * 정산 데이터

## 데이터 불러오기

In [ ]:
cal_df = pd.read_excel('정산DATA_20220601_20230430_map.xlsx')
cal_df.head()

## 데이터 파악

In [ ]:
# 총 row -> 1609개
cal_df.info()

In [ ]:
cal_df.isnull().sum()

## 데이터 전처리

In [ ]:
# 필요한 column 추출 -> 사입업체명, 대납비, 청구금액
cal_df2 = cal_df[['사입업체명', '대납비', '청구금액']]
cal_df2.head()

In [ ]:
# 청구금액이 0인 row -> 26개
cal_df2[cal_df2['청구금액'] == 0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 1 to 1541
Data columns (total 3 columns):
사입업체명    26 non-null object
대납비      26 non-null int64
청구금액     26 non-null int64
dtypes: int64(2), object(1)
memory usage: 832.0+ bytes


In [ ]:
# 청구금액이 0원인 row를 drop -> row 갯수 1583개 (결측치 없음)
cal_df2 = cal_df2.drop(cal_df2[cal_df2['청구금액'] == 0].index)
cal_df2.info()

In [ ]:
# 정산 데이터에서 '사입업체명' 확인해보기
cal_df2['사입업체명'].unique()

In [ ]:
# 주문데이터의 사입업체명과 맞추기 위해 불일치한 사입업체명 E100 삭제(1개의 row)
cal_df2 = cal_df2.drop(cal_df2[cal_df2['사입업체명'] == 'B100'].index)
cal_df2['사입업체명'].unique()

## 데이터 저장

In [ ]:
cal_df2.to_csv('정산데이터_final.csv')